In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
#import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import pickle
from sklearn.model_selection import train_test_split
import torch.optim as optim


In [4]:
#load the shadow model trained in the other python script
device = "cpu"
resnet_shadow =  models.mobilenet_v2(pretrained=False)
num_classes = 200  # Change this to the number of classes in your dataset
num_features = resnet_shadow.classifier[1].in_features
resnet_shadow.classifier = nn.Sequential(
    nn.Linear(num_features, num_classes)
)

resnet_cifar = torch.load("mobilenet_shadow_tinyimage_overtrained.pth",map_location=torch.device('cpu'))
resnet_shadow.load_state_dict(resnet_cifar)

<All keys matched successfully>

In [5]:
DATA_PATH = 'shadow.p'
# Change the DATA_PATH to your local pickle file path

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)


#splitting
#only use train set here
train_data, val_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
dataloader = torch.utils.data.DataLoader(
    train_data, batch_size=
     1 , shuffle=False, num_workers=2)
testloader =  torch.utils.data.DataLoader(val_data, batch_size=1,
                                          shuffle=True, num_workers=2)

for batch_idx, (img, label) in enumerate(dataloader):
    img = img.to(device)


In [6]:
#generate dataset for attack model
resnet_shadow.eval()
dataset_attack = []
with torch.no_grad():
    for images, labels in testloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)
        dataset_attack.append([top_values,0])
        
with torch.no_grad():
    for images, labels in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)

        dataset_attack.append([top_values,1])
        



In [43]:
standardized_data_list = dataset_attack
"""
# Convert all tensors to the same dtype first
tensors = [data[0].float() for data in dataset_attack]  # Ensure all tensors are Float type
all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# Calculate mean and std
mean = all_data.mean(dim=0)
std = all_data.std(dim=0)

# Standardize data in the list
standardized_data_list = [( (data[0] - mean) / std, data[1] ) for data in dataset_attack]
"""

'\n# Convert all tensors to the same dtype first\ntensors = [data[0].float() for data in dataset_attack]  # Ensure all tensors are Float type\nall_data = torch.cat(tensors, dim=0)  # Concatenate all tensors\n\n# Calculate mean and std\nmean = all_data.mean(dim=0)\nstd = all_data.std(dim=0)\n\n# Standardize data in the list\nstandardized_data_list = [( (data[0] - mean) / std, data[1] ) for data in dataset_attack]\n'

In [44]:
dataloader_attack = torch.utils.data.DataLoader(
    
    standardized_data_list, batch_size=64, shuffle=True, num_workers=2) #shuffled training data

In [45]:
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)  # Matches the output of fc1
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)  # Matches the output of fc2
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)  # Matches the output of fc3
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)  # Matches the output of fc4
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)   # Matches the output of fc5
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

# Ensure input dimension is correct


In [46]:
model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [47]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [48]:
# Define your training function

def train(model, dataloader, optimizer, criterion, epochs):
    for inputs, labels in dataloader:
        inputs = inputs.float()  # Ensures input tensors are floats
        labels = labels.float().view(-1, 1)  # Ensures labels are floats and reshaped correctly

        optimizer.zero_grad()
        outputs = model(inputs.squeeze(dim = 1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

In [49]:
model.train()

SimpleNN(
  (fc1): Linear(in_features=3, out_features=32, bias=True)
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (bn4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=16, out_features=8, bias=True)
  (bn5): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)

In [50]:
#dict =  torch.load('attack_model.pth_3', map_location='cpu')
#model.load_state_dict(dict)

In [71]:
model.train()
train(model,dataloader_attack,optimizer,criterion,500)

Loss: 0.6769741773605347
Loss: 0.7068108320236206
Loss: 0.6902546882629395
Loss: 0.6831803917884827
Loss: 0.6851001977920532
Loss: 0.6995765566825867
Loss: 0.725834846496582
Loss: 0.6882588863372803
Loss: 0.6920490264892578
Loss: 0.7127604484558105
Loss: 0.710884690284729
Loss: 0.6780987977981567
Loss: 0.7060308456420898
Loss: 0.6552491188049316
Loss: 0.6817694902420044
Loss: 0.7044413685798645
Loss: 0.687321662902832
Loss: 0.6838076114654541
Loss: 0.6739083528518677
Loss: 0.7122789621353149
Loss: 0.7138328552246094
Loss: 0.6798689365386963
Loss: 0.6819514036178589
Loss: 0.6931749582290649
Loss: 0.6932700276374817
Loss: 0.6718659400939941
Loss: 0.6873555779457092
Loss: 0.6713719367980957
Loss: 0.7053187489509583
Loss: 0.7080845236778259
Loss: 0.6893560290336609
Loss: 0.7067003846168518
Loss: 0.7043976783752441
Loss: 0.7198036909103394
Loss: 0.6660939455032349
Loss: 0.7251268625259399
Loss: 0.7257080078125
Loss: 0.7302195429801941
Loss: 0.6707103848457336
Loss: 0.7094096541404724
Loss: 

In [72]:
torch.save(model.state_dict(), 'attack_mobilenet_tinyimage.pth')

<h1>Evaluation</h1>

In [73]:
DATA_PATH = "eval.p"

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
# Convert all tensors to the same dtype first

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=1 , shuffle=False, num_workers=2)
#splitting
for batch_idx, (img, label, membership) in enumerate(dataloader):
    img = img.to(device)


# Define your evaluation function


In [74]:
dataset = []
resnet_shadow.eval()
with torch.no_grad():
    for images,_, b in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        print(logits)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        

        dataset.append([top_values,b])

tensor([[ -8.2804, -20.8301, -10.9754, -12.1012, -20.1677, -14.4420, -14.7959,
         -13.2221, -17.4829, -14.2656, -13.4200,  -9.2168, -13.5704, -12.6291,
         -15.2590,  -5.3202,  -7.2727, -11.0131, -12.6892, -14.6782,  -7.8624,
          -6.6097, -15.4366, -16.1748,  -9.9049,  -8.0062,  -7.6091,  -5.2623,
          -5.7471,  -6.1890,  -5.6141,  -9.1782,  -9.6883,  -6.4668,  -7.3928,
          -1.2201, -12.2599,  -9.8654,  -5.0058, -16.8974, -15.3551, -12.2440,
         -11.7820, -10.4923, -16.1730, -12.9619, -11.8440,  -4.5346,  -5.1260,
          -0.2146,   2.1040,  -3.6951, -12.7356,  -6.4068,  -8.4584,  -7.1649,
          -4.6542,  -0.7916, -12.3375, -11.3929, -16.7846, -13.4274, -12.5394,
          -5.2597, -13.2222, -14.9132,   0.4915,  -6.2093,  -8.9941,  -7.5603,
         -11.8719,  -9.5157,  -9.2778, -11.8544, -14.2364,  -4.7615,  -5.0763,
         -17.8917,  -5.5906,  -9.2208,  -1.6146, -18.5217, -15.7501, -16.5965,
          -6.5063, -12.0771, -20.1333,  -8.3166,  -7

In [14]:
resnet_target_model =  models.mobilenet_v2(pretrained=False)
num_classes = 200  # Change this to the number of classes in your dataset
num_features = resnet_target_model.classifier[1].in_features
resnet_target_model.classifier = nn.Sequential(
    # nn.Dropout(p=0.0),  # Optional dropout layer for regularization
    nn.Linear(num_features, num_classes)
)
check = torch.load("models/mobilenetv2_tinyimagenet.pth", map_location=torch.device('cpu'))
# Fixing wrong state dict names
check["net"]["classifier.0.weight"] = check["net"]["classifier.1.weight"]
check["net"]["classifier.0.bias"] = check["net"]["classifier.1.bias"]
check["net"].pop("classifier.1.weight")
check["net"].pop("classifier.1.bias")

resnet_target_model.load_state_dict(check["net"])
resnet_target_model.eval()

c:\Users\User-1\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User-1\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [76]:
#get posteriors from target
dataset_eval = []
with torch.no_grad():
    for images,_, member in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_target_model(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values #order poseri
        sorted_tensor, indices = torch.sort(top_values, dim=1,descending=True)
        dataset_eval.append([sorted_tensor, member.item()])
        


In [77]:
"""#onvert all tensors to the same dtype first
tensors = [data[0].float() for data in dataset_eval]  # Ensure all tensors are Float type
all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# Calculate mean and std
mean = all_data.mean(dim=0)
std = all_data.std(dim=0)

# Standardize data in the list
dataset = [( (data[0] - mean) / std, data[1] ) for data in dataset_eval]"""

'#onvert all tensors to the same dtype first\ntensors = [data[0].float() for data in dataset_eval]  # Ensure all tensors are Float type\nall_data = torch.cat(tensors, dim=0)  # Concatenate all tensors\n\n# Calculate mean and std\nmean = all_data.mean(dim=0)\nstd = all_data.std(dim=0)\n\n# Standardize data in the list\ndataset = [( (data[0] - mean) / std, data[1] ) for data in dataset_eval]'

In [78]:
dataloader_eval = torch.utils.data.DataLoader(
    
    dataset, batch_size=1 , shuffle=False, num_workers=2)

In [79]:
def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for data in dataloader:
            inputs = data[0]
            
            labels = data[1].float()
            labels = torch.tensor(labels).float()
            #labels = torch.tensor(labels[0])
            optimizer.zero_grad()

            outputs = model(inputs.squeeze(dim=1))
            predicted = torch.round(outputs)  # Round the outputs to 0 or 1
            #predicted = abs(predicted -1)
            print(predicted,labels)

            total += labels.size(0)  # Increment the total count by batch size
            correct += (predicted == labels).sum().item()  # Count correct predictions

    accuracy = correct / total
    return accuracy

# Assuming you have a DataLoader called 'test_loader' for evaluation
# Create an instance of your model

# Assuming 'test_loader' is your DataLoader for evaluation
accuracy = evaluate(model, dataloader_eval)

print(f'Accuracy: {accuracy:.2f}')

/tmp/ipykernel_39956/100979714.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).float()


tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[1.]]) tensor([[1.]])
tensor([[1.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[1.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[1.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[1.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0.]]) tensor([[1.]])
tensor([[0